### Corporate Finance: Group project #1
# BUILDING A RELATIVE VALUATION ALGORITHM
In this project, we will guide you on how to use Python to create a proper algorithm that outputs crucial information for the relative valuation process.

As you know, Relative Valuation (or Valuation via Multiples) is extremely useful for its simplicity, although this can also entail some problems. The underlying idea for this valuation is that similar firms should be trading (priced) at similar prices with respect to their multiples (e.g., PE, EV/EBITDA, EV/Sales, …)

Even though it is relatively easy to perform these calculations, provided that you have access to the data, we will create an algorithm to facilitate the task.

---
## 0. Import required packages
In this exercise, we will use [pandas](https://pandas.pydata.org/) and [plotly](https://plotly.com/).

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots

---
## 1. Define the universe
We will consider a group of biopharma companies. This list contains their stock market tickers. All of them belong to the SP 500 index.

In [2]:
UNIVERSE = ['ABBV-US', 'AMGN-US', 'ABT-US', 'JNJ-US', 'MMM-US', 'MRK-US', 'PFE-US', 'TMO-US']

---
## 2. Read the SP 500 constituents
The excel file `listing.xlsx` constains the constituents of the SP500 including some company-level data we will need later.

In [4]:
listing = pd.read_excel('listing.xlsx', index_col='Id')
listing = listing.set_index('Symbol')
listing.tail()

,Date,Exchange,Security Type,Company Name,RBICS Economy,RBICS Sector,RBICS Subsector,Industry Group,Industry,Subindustry,Common Shares Outstanding - Security Level,Shares - Company Level,Market Value - Company Level
Symbol,,,,,,,,,,,,,
PARA-US,2022-09-16,NASDAQ,SHARE,Paramount Global Class B,Consumer Services,Media and Publishing Services,Media and Publishing Services,Entertainment and Programming Providers,Other Entertainment and Programming Providers,General Entertainment and Programming,608.42145,649.12690,15090.597279
CARR-US,2022-09-16,NYSE,SHARE,Carrier Global Corp.,Industrials,Industrial Manufacturing,Machinery Manufacturing,"Air, Liquid and Gas Control Equipment",Environmental Control Machinery/Equipment Prod...,"Heating, Ventilation and Air Conditioning Prod...",841.58344,841.58344,33999.972061
OTIS-US,2022-09-16,NYSE,SHARE,Otis Worldwide Corporation,Business Services,Business Services,Business Support Services,Facilities and Other Support Services,Facilities Support Services,Building Maintenance and Engineering Services,422.79443,422.79443,29143.220437
MTCH-US,2022-09-16,NASDAQ,SHARE,"Match Group, Inc.",Technology,Software and Consulting,Internet and Data Services,Consumer Data and Services,Information and News Content Providers and Sites,Other Classifieds and Directories Media and Sites,282.98645,282.98645,16373.596181
WBD-US,2022-09-16,NASDAQ,SHARE,"Warner Bros. Discovery, Inc. Series A",Consumer Services,Media and Publishing Services,Media and Publishing Services,Entertainment and Programming Providers,Television Cable and Broadcast Networks,US TV Cable and Broadcast Networks,2427.59280,2427.59280,31971.397011


---
## 3. Sunburst chart of industry and market capitalization
Let's generate a [sunburst chart](https://plotly.com/python/sunburst-charts/) of the SP 500 by industry and market capitalization. It is interactive!

In [8]:
fig = px.sunburst(listing,
                  path=['RBICS Economy', 'RBICS Sector'], 
                  values='Market Value - Company Level', 
                  width=800, height=800)
fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

---
## 4. Read stock prices
The excel `prices.xlsx` contains daily price, return and volume data.

In [ ]:
all_prices = pd.read_excel('prices.xlsx', index_col='Id')
all_prices.tail(3)

---
## 5. Extract prices for a single company
The table contains data in long-format that belongs to many companies at different dates. How can we extract the prices of a given company?  
Remember to set an index that is unique and representative of each row (the date).

In [ ]:
# function
def get_prices(all_prices, ticker):
    df = all_prices[all_prices['symbol'] == ticker]
    df = df.set_index('date')
    return df

# example: retrieve Johnson & Johnson price data
df = get_prices(all_prices, 'JNJ-US')
df.tail()

---
## 6. Plot prices and volume of a company
Let's display in a single chart the prices and volume of a given company.  
The price chart is a [scatter plot](https://plotly.com/python/line-and-scatter/) and volume is a [bar plot](https://plotly.com/python/bar-charts/).

In [ ]:
# function
def plot_prices(prices, title):
    # Create subplots, that is, one chart on top of another
    fig = make_subplots(rows=2, cols=1,
                        shared_xaxes=True, 
                        vertical_spacing=0.03, subplot_titles=('Adjusted Price', 'Volume'), 
                        row_width=[0.2, 0.7])

    # top chart: prices
    fig.add_trace(go.Scatter(x = prices.index, y = prices['Adjusted Price'], name='Price'), 
                    row=1, col=1)

    # bottom chart: bar plot for volumes
    fig.add_trace(go.Bar(x=prices.index, y=prices['Volume'], marker_line_color='red', name='Volume'), row=2, col=1)

    # Do not show rangeslider plot 
    fig.update_layout(xaxis_rangeslider_visible=False, width=1200, height=600, title=title)
    fig.show()

# example: plot johnson & johnson prices and volume
# df is the DataFrame retrieved in the cell above
plot_prices(df, 'Prices of Johnson & Johnson')

---
## 7. Read Fundamental Data
Let's read the excel files containing the balance sheets, income statements and cash flows.  
(Cash flows are not used in the exercise).

In [ ]:
all_balance_sheets = pd.read_excel('balance_sheets.xlsx', index_col='Id')
all_income_statements = pd.read_excel('income_statements.xlsx', index_col='Id')
all_cash_flows = pd.read_excel('cash_flows.xlsx', index_col='Id')

---
## 8. Multiples: DO YOUR MAGIC HERE!!
Insert below all the code you need to calculate the multiples!

$$PE=\frac{MarketCap}{Earnings_{TTM}}$$

$$\frac{EV}{EBITDA}=\frac{MarketCap + Debt - Cash}{EBITDA_{TTM}}$$

In [ ]:
###
# YOUR CODE HERE
###

---
## 9. The visualization code, as promised
Given a `DataFrame` with the correct format and structure, this function will display:
- High band
- Median
- Low band
- The multiple(s) of the company(ies) selected in the legend on the right.

In [ ]:
def plot_multiple(name, universe, data):

    fig = go.Figure()
    
    fig.add_trace(
        go.Scatter(x = data.index,
                   y = data['median'],
                   line = {'dash': 'solid', 'width':5},
                   line_color = 'gray',
                   name = 'median',
                   opacity = 0.25)
    )
    
    for ticker in universe:
        fig.add_trace(
            go.Scatter(x = data.index,
                       y = data[ticker],
                       #line_color = 'red',
                       visible='legendonly',
                       name = ticker)
        )

    # Upper Bound
    fig.add_trace(
        go.Scatter(x = data.index,
                   y = data['high'],
                   line_color = 'lightgray',
                   line = {'dash': 'dot', 'width':1},                   
                   name = 'upper band',
                   opacity = 0.25)
    )

    # Lower Bound fill in between with parameter 'fill': 'tonexty'
    fig.add_trace(
        go.Scatter(x = data.index,
                   y = data['low'],
                   line_color = 'lightgray',
                   line = {'dash': 'dot', 'width':1},
                   fill = 'tonexty',
                   name = 'lower band',
                   opacity = 0.25)
    )

    # use percentages in axes
    fig.update_layout(
        width=1200,
        height=600,
        title=name,
        xaxis_title='time',
        yaxis_title=name,
    )

    fig.show()

Plot multiple: $\frac{EV}{EBITDA}$

In [ ]:
# example
#plot_multiple('EV/EBITDA', UNIVERSE, all_ev_ebidta)

Plot multiple: $PE=\frac{Price}{Earnings}$

In [ ]:
# example
#plot_multiple('PE', UNIVERSE, all_pe)